# Drawing Graph Edges Using `geomSegment()` and `geomCurve()`

Beyond merely connecting two points on a chart, the `geomSegment()` and `geomCurve()` geometries</br>
can, with some fine-tuning, help to visualize graph-like data.

Use aesthetics `sizeStart/End` and `strokeStart/End` to allow `segment/curve`</br>
to take into account the size of the point from which it starts/ends and to avoid drawing over it.

Utilize the `spacer` parameter for further manual fine-tuning.

In [1]:
%useLatestDescriptors
%use lets-plot
%use dataframe

In [2]:
LetsPlot.getInfo()

Lets-Plot Kotlin API v.4.7.2. Frontend: Notebook with dynamically loaded JS. Lets-Plot JS v.4.3.2.

In [3]:
val x = listOf(-1, 0, 1)
val y = listOf(-1, 1, -1)
val shape = listOf(1, 16, 21)
val size = listOf(1, 2, 3)
val stroke = listOf(1, 0, 2)

val verticesData = mapOf(
    "x" to x,
    "y" to y,
    "shape" to shape,
    "size" to size,
    "stroke" to stroke
)

In [4]:
val verticesLayer = geomPoint(color = "#4575b4", fill = "#abd9e9") {
    x = "x"; y = "y"; size = "size"; shape = "shape"; stroke = "stroke"
}

In [5]:
val graphVertices =
    letsPlot(verticesData) + verticesLayer + 
    scaleSize(range = 20 to 30, guide = "none") + 
    scaleStroke(range = 0 to 10, guide = "none") +
    scaleShapeIdentity() +
    lims(x = -1.5 to 1.5, y = -1.5 to 1.5)

graphVertices

In [6]:
val edgesData = mapOf(
    "xEnd" to x.toMutableList().apply { this.add(this.removeFirst()) },
    "yEnd" to y.toMutableList().apply { this.add(this.removeFirst()) }
)

#### 1. Draw Ugly Graph Edges

In [7]:
val uglyEdges = geomSegment(data = edgesData, arrow = arrow(ends = "both")) {
    x = "x"; y = "y"; xend = "xEnd"; yend = "yEnd"
} 

graphVertices + uglyEdges

#### 2. Draw Nice Graph Edges

In [8]:
// Append an info on the sizes of vertices in the graph.
val edgesData2 = edgesData + mapOf(
    "sizeEnd" to size.toMutableList().apply { this.add(this.removeFirst()) },
    "strokeEnd" to stroke.toMutableList().apply { this.add(this.removeFirst()) }
)

In [9]:
// Use `segment` and then `curve` to draw "nice" edges.

val niceEdgesS = 
    geomSegment(
        data = edgesData2,       
        arrow = arrow(ends = "both"),
        spacer = 5                                       // New! Add a "spacer".
    ) {
        x = "x"; y = "y"; xend = "xEnd"; yend = "yEnd";
        sizeStart = "size"; sizeEnd = "sizeEnd";         // New! Take into account sizes of points 
                                                         // connected by the edge.
        strokeStart = "stroke"; strokeEnd = "strokeEnd"  // New! Take into account stroke (width) of points 
                                                         // connected by the edge.
    }

val niceEdgesC = geomCurve(
        data = edgesData2,       
        arrow = arrow(ends = "both"),
        curvature = -0.3,
        spacer = 5
    ) {
        x = "x"; y = "y"; xend = "xEnd"; yend = "yEnd";
        sizeStart = "size"; sizeEnd = "sizeEnd";
        strokeStart = "stroke"; strokeEnd = "strokeEnd"
    }

gggrid(listOf(
    graphVertices + niceEdgesS,
    graphVertices + niceEdgesC
))

#### 3. Another Example of Graph Visualization

In [10]:
val nodesDf = dataFrameOf(
    "node" to listOf("Living\nThings", "Animals", "Plants", "Dogs", "Cows", "Herbs"),
    "x" to listOf(0, -1, 1, -2, 0, 2),
    "y" to listOf(1, 0, 0, -1, -1, -1)
)

var edgesDf = dataFrameOf(
    "from" to listOf("Animals", "Plants", "Dogs", "Cows", "Cows", "Herbs"),
    "to" to listOf("Living\nThings", "Living\nThings", "Animals", "Animals", "Herbs", "Plants"),
    "relation" to listOf("is", "is", "is", "is", "eat", "is")
)

In [11]:
edgesDf = edgesDf.join(nodesDf) { to match right.node }.rename(Pair("x", "xTo")).rename(Pair("y", "yTo"))
edgesDf = edgesDf.join(nodesDf) { from match right.node }
edgesDf

from,to,relation,xTo,yTo,x,y
Animals,Living Things,is,0,1,-1,0
Plants,Living Things,is,0,1,1,0
Dogs,Animals,is,-1,0,-2,-1
Cows,Animals,is,-1,0,0,-1
Cows,Herbs,eat,2,-1,0,-1
Herbs,Plants,is,1,0,2,-1


In [12]:
letsPlot(nodesDf.toMap()) { x = "x"; y = "y" } +
    geomSegment(data = edgesDf.toMap(), sizeEnd = 25, arrow = arrow()) { 
        x = "x"; y = "y"; xend = "xTo"; yend = "yTo"; color = "relation" 
    } +
    geomPoint(color = "#2166ac", fill = "#d1e5f0", shape = 21, size = 25) +
    scaleColorManual(listOf("#2166ac", "#d6604d")) +
    geomText() { label = "node" } +
    coordCartesian(-3 to 3, -1.5 to 1.5) +
    themeVoid()